In [ ]:
import pandas as pd
price_column = 'Vážený průměr cen (EUR/MWh)'
df = pd.read_excel("data/OTE_2024.xlsx", sheet_name="VDT (EUR)", skiprows=5, usecols="A:L")

#     # 2. Принудительно конвертируем колонку с ценой в числовой формат
#     # Любой текст ('N/A', '-') превратится в NaN (маркер отсутствия данных)
# df[price_column] = pd.to_numeric(df[price_column], errors='coerce')

#     # 3. Создаем временной столбец
# start_time = df['Časový interval'].str.split('-').str[0]
# df['datetime'] = pd.to_datetime(df['Den'].astype(str) + ' ' + start_time)

#     # 4. Устанавливаем 'datetime' как индекс (ключевой шаг)
# df.set_index('datetime', inplace=True)
    
#     # 5. Удаляем ненужные столбцы
# df.drop(columns=['Den', 'Časový interval'], inplace=True)

#     # 6. Сортируем и ОЧИЩАЕМ данные от NaN
# df.sort_index(inplace=True)
# df.drop_duplicates(inplace=True)
    
#     # fillna заполнит пропуски, которые появились после to_numeric
# df.fillna(method='ffill', inplace=True)
# df.fillna(method='bfill', inplace=True) # Заполняем в обратную сторону на случай, если NaN в самом начале
    
#     # 7. Фильтруем данные по нужному диапазону дат
# # start_date = '2025-01-01 00:00:00'
# # end_date = '2025-07-31 23:59:59' # Убедитесь, что ваш диапазон корректен
# # vdt2025 = df.loc[start_date:end_date].copy()




#############################################################################################################

df['datetime'] = pd.to_datetime(df['Den'])
df['datetime'] = df['datetime'] + pd.to_timedelta(df['Hodina'] - 1, unit='h')

df.sort_values('datetime', inplace=True)
df.drop_duplicates(subset=['datetime'], keep='first', inplace=True)

vdt2024 = df[['datetime', 'Vážený průměr cen (EUR/MWh)']]
vdt2024.set_index('datetime', inplace=True)

vdt_15min = vdt2024.resample('15T').ffill()

vdt_15min.reset_index(inplace=True)
vdt_15min['Den'] = vdt_15min['datetime'].dt.normalize()
vdt_15min['Perioda'] = vdt_15min.groupby(vdt_15min['datetime'].dt.date).cumcount() + 1

vdt2024_formatted = vdt_15min[['datetime', 'Den', 'Perioda', 'Vážený průměr cen (EUR/MWh)']]
vdt2024_formatted.fillna(method='ffill', inplace=True)


VDT = vdt2024_formatted[vdt2024_formatted['datetime'].dt.year == 2024]

VDT.set_index('datetime', inplace=True)

In [1]:
import pandas as pd
VDT = (pd.read_excel('data/data.xlsx', sheet_name='VDT (R2W)')).set_index('datetime', inplace=False)
VDT = VDT[VDT.index <= '2025-06-30']
VDT['Vážený průměr cen (EUR/MWh)'].fillna(method='ffill', inplace=True)

C:\Users\Tamerlan.Ismakatov\AppData\Local\Temp\ipykernel_20828\3680577874.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  VDT['Vážený průměr cen (EUR/MWh)'].fillna(method='ffill', inplace=True)
C:\Users\Tamerlan.Ismakatov\AppData\Local\Temp\ipykernel_20828\3680577874.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  VDT['Vážený průměr cen (EUR/MWh)'].fillna(method='ffill', inplace=True)


In [2]:
from BessOptimizer import analyze_projects, BESS_Optimizer, export_full_report_to_excel

PROJECTS_CONFIG = {
    'pribor': {
        'name': 'pribor',
        'bess_power_mw': 2.5,
        'bess_capacity_mwh': 5.6,
        'export_limit_mw': 2.5,      
        'import_limit_mw': 2.5,      
        'efficiency': 0.85,
        'max_cycles': 2          
    }
}   

results = analyze_projects(VDT, projects_config=PROJECTS_CONFIG,analysis_period_days=365, show_daily_reports=False)


Successfully analyzed 364 days
Annual BESS income: 285,985 EUR


In [3]:
for i in results.keys():
    optimizer_instance = BESS_Optimizer(PROJECTS_CONFIG[i])

    project_key_to_export = i
    project_data_to_export = results[project_key_to_export]
            
    excel_filename = f"full_report_{project_key_to_export}.xlsx"
            
    export_full_report_to_excel(project_data_to_export,VDT, optimizer_instance, excel_filename, with_plots=False)


Creating Excel report 'full_report_pribor.xlsx' for project 'pribor'...
✅ Report 'full_report_pribor.xlsx' was successfully created!
